In [6]:
import nltk 
import pandas as pd 
import numpy as np 

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression 
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords 

In [7]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

positive_reviews = BeautifulSoup(open('/Users/mac/Downloads/sorted_data_acl/electronics/positive.review').read())
positive_reviews = positive_reviews.findAll('review_text')

negative_reviews = BeautifulSoup(open('/Users/mac/Downloads/sorted_data_acl/electronics/negative.review'))
negative_reviews = negative_reviews.findAll('review_text')

In [9]:
#balance the data 
np.random.shuffle(positive_reviews)
positive_reviews = positive_reviews[:len(negative_reviews)]

In [13]:
def my_tokenizer(s): 

    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens


word_index_map = {}
current_index = 0 

positive_tokenized= []
negative_tokenized=[]

for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map: 
            word_index_map[token]= current_index
            current_index+=1

for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map: 
            word_index_map[token]= current_index
            current_index+=1

In [14]:
def tokens_to_vector(tokens,label):
    x = np.zeros(len(word_index_map)+1)
    for t in tokens:
        i = word_index_map[t]
        x[i]+= 1
    x=x/x.sum()
    x[-1] = label 
    return x

N = len(positive_tokenized) + len(negative_tokenized)
# (N x D+1 matrix - keeping them together for now so we can shuffle more easily later
data = np.zeros((N, len(word_index_map) + 1))
i = 0
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1
    
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [18]:

# shuffle the data and create train/test splits
# try it multiple times!
np.random.shuffle(data)

X = data[:,:-1]
Y = data[:,-1]

# last 100 rows will be test
Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

In [19]:

model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print("Train accuracy:", model.score(Xtrain, Ytrain))
print("Test accuracy:", model.score(Xtest, Ytest))
print(Classificationrate)

Train accuracy: 0.783157894736842
Test accuracy: 0.82


In [22]:
# let's look at the weights for each word
# try it with different threshold values!
threshold = 0.5
for word, index in word_index_map.iteritems():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

AttributeError: 'dict' object has no attribute 'iteritems'